# How to migrate from legacy LangChain agents to LangGraph

Here we focus on how to move from legacy LangChain agents to LangGraph agents.
LangChain agents (the
[AgentExecutor](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent.AgentExecutor.html#langchain.agents.agent.AgentExecutor)
in particular) have multiple configuration parameters. In this notebook we will
show how those parameters map to the LangGraph
[react agent executor](https://langchain-ai.github.io/langgraph/reference/prebuilt/#create_react_agent).

For more information on how to build agentic workflows in LangGraph, check out
the [docs here](https://langchain-ai.github.io/langgraphjs/how-tos/).

#### Prerequisites

This how-to guide uses Claude (model: "claude-3-haiku-20240307") as the LLM. Set
your Anthropic API key to run.

In [ ]:
// process.env.ANTHROPIC_API_KEY = "sk-...";

// Optional, add tracing in LangSmith
// process.env.LANGCHAIN_API_KEY = "ls...";
process.env.LANGCHAIN_CALLBACKS_BACKGROUND = "true";
process.env.LANGCHAIN_TRACING_V2 = "true";
process.env.LANGCHAIN_PROJECT = "How to migrate: LangGraphJS";

## Basic Usage

For basic creation and usage of a tool-calling ReAct-style agent, the
functionality is the same. First, let's define a model and tool(s), then we'll
use those to create an agent.


In [1]:
import { DynamicStructuredTool } from "@langchain/core/tools";
import { z } from "zod";
import { ChatAnthropic } from "@langchain/anthropic";

const llm = new ChatAnthropic({ model: "claude-3-haiku-20240307" });

const magicTool = new DynamicStructuredTool({
  name: "magic_function",
  description: "Applies a magic function to an input.",
  schema: z.object({
    input: z.number(),
  }),
  func: async ({ input }: { input: number }) => {
    return `${input + 2}`;
  },
});

const tools = [magicTool];

const query = "what is the value of magic_function(3)?";


For the LangChain
[AgentExecutor](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent.AgentExecutor.html#langchain.agents.agent.AgentExecutor),
we define a prompt with a placeholder for the agent's scratchpad. The agent can
be invoked as follows:


In [2]:
import {
  ChatPromptTemplate,
  MessagesPlaceholder,
} from "@langchain/core/prompts";
import { createToolCallingAgent } from "langchain/agents";
import { AgentExecutor } from "langchain/agents";

const prompt = ChatPromptTemplate.fromMessages([
  ["system", "You are a helpful assistant"],
  ["human", `{input}`],
  new MessagesPlaceholder("agent_scratchpad"),
]);

const agent = createToolCallingAgent({ llm: llm, tools: tools, prompt });
const agentExecutor = new AgentExecutor({ agent, tools: tools });

await agentExecutor.invoke({ input: query });


{
  input: "what is the value of magic_function(3)?",
  output: "So the value of magic_function(3) is 5."
}

LangGraph's
[react agent executor](https://langchain-ai.github.io/langgraph/reference/prebuilt/#create_react_agent)
manages a state that is defined by a list of messages. It will continue to
process the list until there are no tool calls in the agent's output. To kick it
off, we input a list of messages. The output will contain the entire state of
the graph-- in this case, the conversation history.


In [3]:
import { createReactAgent } from "@langchain/langgraph/prebuilt";
import { HumanMessage } from "@langchain/core/messages";

const app = createReactAgent({ llm, tools });

let agentOutput = await app.invoke({ messages: [new HumanMessage(query)] });

In [4]:
const messageHistory = agentOutput.messages;
const newQuery = "Pardon?";

agentOutput = await app.invoke({
  messages: [...messageHistory, new HumanMessage(newQuery)],
});


{
  messages: [
    HumanMessage {
      lc_serializable: true,
      lc_kwargs: {
        content: "what is the value of magic_function(3)?",
        additional_kwargs: {},
        response_metadata: {}
      },
      lc_namespace: [ "langchain_core", "messages" ],
      content: "what is the value of magic_function(3)?",
      name: undefined,
      additional_kwargs: {},
      response_metadata: {}
    },
    AIMessage {
      lc_serializable: true,
      lc_kwargs: {
        content: [ [Object], [Object] ],
        additional_kwargs: {
          id: "msg_01CykT2w3Wgp4tP4MRET3S8j",
          type: "message",
          role: "assistant",
          model: "claude-3-haiku-20240307",
          stop_sequence: null,
          usage: [Object],
          stop_reason: "tool_use"
        },
        tool_calls: [ [Object] ],
        invalid_tool_calls: [],
        response_metadata: {}
      },
      lc_namespace: [ "langchain_core", "messages" ],
      content: [
        {
          type: "te

## Prompt Templates

With legacy LangChain agents you have to pass in a prompt template. You can use
this to control the agent.

With LangGraph
[react agent executor](https://langchain-ai.github.io/langgraph/reference/prebuilt/#create_react_agent),
by default there is no prompt. You can achieve similar control over the agent in
a few ways:

1. Pass in a system message as input
2. Initialize the agent with a system message
3. Initialize the agent with a function to transform messages before passing to
   the model.

Let's take a look at all of these below. We will pass in custom instructions to
get the agent to respond in Spanish.

First up, using AgentExecutor:


In [5]:
const spanishPrompt = ChatPromptTemplate.fromMessages([
  ["system", "You are a helpful assistant. Respond only in Spanish."],
  ["human", `{input}`],
  new MessagesPlaceholder("agent_scratchpad"),
]);

const spanishAgent = createToolCallingAgent({
  llm: llm,
  tools: tools,
  prompt: spanishPrompt,
});
const spanishAgentExecutor = new AgentExecutor({
  agent: spanishAgent,
  tools: tools,
});

await spanishAgentExecutor.invoke({ input: query });


{
  input: "what is the value of magic_function(3)?",
  output: "Por lo tanto, el resultado de aplicar la función mágica al valor 3 es 5."
}

Now, let's pass a custom system message to
[react agent executor](https://langchain-ai.github.io/langgraph/reference/prebuilt/#create_react_agent).
This can either be a string or a LangChain SystemMessage.


In [12]:
import { SystemMessage } from "@langchain/core/messages";

const systemMessage = "You are a helpful assistant. Respond only in Spanish.";

// This could also be a SystemMessage object
// const systemMessage = new SystemMessage("You are a helpful assistant. Respond only in Spanish.");

const appWithSystemMessage = createReactAgent({
  llm,
  tools,
  messageModifier: systemMessage,
});

agentOutput = await appWithSystemMessage.invoke({
  messages: [new HumanMessage(query)],
});
agentOutput.messages[agentOutput.messages.length - 1];

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "El valor de invocar magic_function(3) es 5.",
    tool_calls: [],
    invalid_tool_calls: [],
    additional_kwargs: {
      id: "msg_01Tjm4DqarRZLakrxHBEpXF4",
      type: "message",
      role: "assistant",
      model: "claude-3-haiku-20240307",
      stop_sequence: null,
      usage: { input_tokens: 444, output_tokens: 19 },
      stop_reason: "end_turn"
    },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "El valor de invocar magic_function(3) es 5.",
  name: undefined,
  additional_kwargs: {
    id: "msg_01Tjm4DqarRZLakrxHBEpXF4",
    type: "message",
    role: "assistant",
    model: "claude-3-haiku-20240307",
    stop_sequence: null,
    usage: { input_tokens: 444, output_tokens: 19 },
    stop_reason: "end_turn"
  },
  response_metadata: {
    id: "msg_01Tjm4DqarRZLakrxHBEpXF4",
    model: "claude-3-haiku-20240307",
    stop_sequence: null,
    usage: { input_tokens:

We can also pass in an arbitrary function. This function should take in a list
of messages and output a list of messages. We can do all types of arbitrary
formatting of messages here. In this cases, let's just add a SystemMessage to
the start of the list of messages.


In [13]:
import { BaseMessage, SystemMessage } from "@langchain/core/messages";

const modifyMessages = (messages: BaseMessage[]) => {
  return [
    new SystemMessage("You are a helpful assistant. Respond only in Spanish."),
    ...messages,
    new HumanMessage("Also say 'Pandamonium!' after the answer."),
  ];
};

const appWithMessagesModifier = createReactAgent({
  llm,
  tools,
  messageModifier: modifyMessages,
});

agentOutput = await appWithMessagesModifier.invoke({
  messages: [new HumanMessage(query)],
});

console.log({
  input: query,
  output: agentOutput.messages[agentOutput.messages.length - 1].content,
});

{
  input: "what is the value of magic_function(3)?",
  output: "5. ¡Pandamonium!"
}


## Memory

With LangChain's
[AgentExecutor](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent.AgentExecutor.html#langchain.agents.agent.AgentExecutor.iter),
you could add chat
[Memory](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent.AgentExecutor.html#langchain.agents.agent.AgentExecutor.memory)
so it can engage in a multi-turn conversation.


In [14]:
import { ChatMessageHistory } from "@langchain/community/stores/message/in_memory";
import { RunnableWithMessageHistory } from "@langchain/core/runnables";

const memory = new ChatMessageHistory();
const agentExecutorWithMemory = new RunnableWithMessageHistory({
  runnable: agentExecutor,
  getMessageHistory: () => memory,
  inputMessagesKey: "input",
  historyMessagesKey: "chat_history",
});

const config = { configurable: { sessionId: "test-session" } };

agentOutput = await agentExecutorWithMemory.invoke(
  { input: "Hi, I'm polly! What's the output of magic_function of 3?" },
  config,
);
console.log(agentOutput.output);
agentOutput = await agentExecutorWithMemory.invoke(
  { input: "Remember my name?" },
  config,
);
console.log("---");
console.log(agentOutput.output);
console.log("---");
agentOutput = await agentExecutorWithMemory.invoke(
  { input: "what was that output again?" },
  config,
);
console.log(agentOutput.output);

The magic_function applied to the input of 3 outputs the value 5.
---
I'm afraid I don't actually have the capability to remember your name. As an AI assistant, I don't have a persistent memory of our conversation or personal details about you. I respond based on the current context of our interaction. Could you please restate your request so I can try to assist you further?
---
I don't have any previous output to reference. Could you please provide more context about what you are asking about? I'd be happy to assist once I understand the specific request you have.


#### In LangGraph

Memory is just
[persistence](https://langchain-ai.github.io/langgraph/how-tos/persistence/),
aka
[checkpointing](https://langchain-ai.github.io/langgraph/reference/checkpoints/).

Add a `checkpointer` to the agent and you get chat memory for free.


In [16]:
import { MemorySaver } from "@langchain/langgraph";

const memory = new MemorySaver();
const appWithMemory = createReactAgent({ llm, tools, checkpointSaver: memory });

const config = { configurable: { thread_id: "test-thread" } };

agentOutput = await appWithMemory.invoke(
  {
    messages: [
      new HumanMessage(
        "Hi, I'm polly! What's the output of magic_function of 3?",
      ),
    ],
  },
  config,
);
console.log(agentOutput.messages[agentOutput.messages.length - 1].content);
console.log("---");
agentOutput = await appWithMemory.invoke(
  { messages: [new HumanMessage("Remember my name?")] },
  config,
);
console.log(agentOutput.messages[agentOutput.messages.length - 1].content);
console.log("---");
agentOutput = await appWithMemory.invoke(
  { messages: [new HumanMessage("what was that output again?")] },
  config,
);
console.log(agentOutput.messages[agentOutput.messages.length - 1].content);

The magic_function applied to the input of 3 returns the output of 5.
---
It's nice to meet you, Polly! I'll be sure to remember your name.
---
[]


## Iterating through steps

With LangChain's
[AgentExecutor](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent.AgentExecutor.html#langchain.agents.agent.AgentExecutor.iter),
you could iterate over the steps using the
[stream](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable.stream)
(or async `astream`) methods or the
[iter](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent.AgentExecutor.html#langchain.agents.agent.AgentExecutor.iter)
method. LangGraph supports stepwise iteration using
[stream](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable.stream)


In [17]:
for await (const step of await agentExecutor.stream({ input: query })) {
  console.log(step);
}


{
  intermediateSteps: [
    {
      action: {
        tool: "magic_function",
        toolInput: { input: 3 },
        toolCallId: "toolu_01FXcVYzTS9NH9urhUcL3jmw",
        log: 'Invoking "magic_function" with {"input":3}\n' +
          `[{"type":"text","text":"Okay, let's use the magic_functio`... 156 more characters,
        messageLog: [ [AIMessageChunk] ]
      },
      observation: "5"
    }
  ]
}
{
  output: "The magic_function tool takes an input number and returns a transformed value. When we call magic_fu"... 34 more characters
}


#### In LangGraph

In LangGraph, things are handled natively using
[stream](https://langchain-ai.github.io/langgraph/reference/graphs/#langgraph.graph.graph.CompiledGraph.stream)
or the asynchronous `astream` method.


In [18]:
for await (
  const step of await app.stream(
    { messages: [new HumanMessage(query)] },
    { streamMode: "updates" },
  )
) {
  console.log(step);
}

{
  agent: {
    messages: [
      AIMessage {
        lc_serializable: true,
        lc_kwargs: {
          content: [Array],
          additional_kwargs: [Object],
          tool_calls: [Array],
          invalid_tool_calls: [],
          response_metadata: {}
        },
        lc_namespace: [ "langchain_core", "messages" ],
        content: [ [Object], [Object] ],
        name: undefined,
        additional_kwargs: {
          id: "msg_01CYJghzEr6kx35rWfHc8PMo",
          type: "message",
          role: "assistant",
          model: "claude-3-haiku-20240307",
          stop_sequence: null,
          usage: [Object],
          stop_reason: "tool_use"
        },
        response_metadata: {
          id: "msg_01CYJghzEr6kx35rWfHc8PMo",
          model: "claude-3-haiku-20240307",
          stop_sequence: null,
          usage: [Object],
          stop_reason: "tool_use"
        },
        tool_calls: [ [Object] ],
        invalid_tool_calls: []
      }
    ]
  }
}
{
  tools: {
    me

## `return_intermediate_steps`

Setting this parameter on AgentExecutor allows users to access
intermediate_steps, which pairs agent actions (e.g., tool invocations) with
their outcomes.


In [19]:
const agentExecutorWithIntermediateSteps = new AgentExecutor({
  agent: agent,
  tools: tools,
  returnIntermediateSteps: true,
});

const result = await agentExecutorWithIntermediateSteps.invoke({
  input: query,
});

console.log(result.intermediateSteps);


[
  {
    action: {
      tool: "magic_function",
      toolInput: { input: 3 },
      toolCallId: "toolu_017Q69gSfYvkXSNus9mU8wrL",
      log: 'Invoking "magic_function" with {"input":3}\n' +
        `[{"type":"text","text":"Okay, let's use the magic_functio`... 151 more characters,
      messageLog: [
        AIMessageChunk {
          lc_serializable: true,
          lc_kwargs: [Object],
          lc_namespace: [Array],
          content: [Array],
          name: undefined,
          additional_kwargs: [Object],
          response_metadata: {},
          tool_calls: [Array],
          invalid_tool_calls: [],
          tool_call_chunks: [Array]
        }
      ]
    },
    observation: "5"
  }
]


By default the
[react agent executor](https://langchain-ai.github.io/langgraph/reference/prebuilt/#create_react_agent)
in LangGraph appends all messages to the central state. Therefore, it is easy to
see any intermediate steps by just looking at the full state.


In [20]:
agentOutput = await app.invoke({ messages: [new HumanMessage(query)] });

console.log(agentOutput.messages);


[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "what is the value of magic_function(3)?",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "what is the value of magic_function(3)?",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: [
        {
          type: "text",
          text: "Here is the result of calling the magic_function with an input of 3:"
        },
        {
          type: "tool_use",
          id: "toolu_01VWunWMK2tDtUpeaBhi9qfY",
          name: "magic_function",
          input: [Object]
        }
      ],
      additional_kwargs: {
        id: "msg_01ApEdDDyr54PsqZvzYy6Mhv",
        type: "message",
        role: "assistant",
        model: "claude-3-haiku-20240307",
        stop_sequence: null,
        usage: { input_tokens: 365, output_tokens: 71 },
     

## `max_iterations`

`AgentExecutor` implements a `max_iterations` parameter, whereas this is
controlled via `recursion_limit` in LangGraph.

Note that in AgentExecutor, an "iteration" includes a full turn of tool
invocation and execution. In LangGraph, each step contributes to the recursion
limit, so we will need to multiply by two (and add one) to get equivalent
results.

If the recursion limit is reached, LangGraph raises a specific exception type,
that we can catch and manage similarly to AgentExecutor.


In [21]:
const badMagicTool = new DynamicStructuredTool({
  name: "magic_function",
  description: "Applies a magic function to an input.",
  schema: z.object({
    input: z.string(),
  }),
  func: async ({ input }) => {
    return "Sorry, there was an error. Please try again.";
  },
});

const badTools = [badMagicTool];

const spanishAgentExecutorWithMaxIterations = new AgentExecutor({
  agent: createToolCallingAgent({
    llm: llm,
    tools: badTools,
    prompt: spanishPrompt,
  }),
  tools: badTools,
  verbose: true,
  maxIterations: 2,
});

await spanishAgentExecutorWithMaxIterations.invoke({ input: query });


[chain/start] [1:chain:AgentExecutor] Entering Chain run with input: {
  "input": "what is the value of magic_function(3)?"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:ToolCallingAgent] Entering Chain run with input: {
  "input": "what is the value of magic_function(3)?",
  "steps": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:ToolCallingAgent > 3:chain:RunnableAssign] Entering Chain run with input: {
  "input": ""
}
[chain/start] [1:chain:AgentExecutor > 2:chain:ToolCallingAgent > 3:chain:RunnableAssign > 4:chain:RunnableMap] Entering Chain run with input: {
  "input": ""
}
[chain/start] [1:chain:AgentExecutor > 2:chain:ToolCallingAgent > 3:chain:RunnableAssign > 4:chain:RunnableMap > 5:chain:RunnableLambda] Entering Chain run with input: {
  "input": ""
}
[chain/end] [1:chain:AgentExecutor > 2:chain:ToolCallingAgent > 3:chain:RunnableAssign > 4:chain:RunnableMap > 5:chain:RunnableLambda] s] Exiting Chain run with output: {
  "output": []
}
[chain/end] [1:chain:AgentExecut

{
  input: "what is the value of magic_function(3)?",
  output: 'Lo siento, pero la herramienta "magic_function" espera un parámetro de tipo "string", no un número e'... 112 more characters
}

In [22]:
import { GraphRecursionError } from "@langchain/langgraph";

const RECURSION_LIMIT = 2 * 2 + 1;

const appWithBadTools = createReactAgent({ llm, tools: badTools });

try {
  await appWithBadTools.invoke({ messages: [new HumanMessage(query)] }, {
    recursionLimit: RECURSION_LIMIT,
  });
} catch (e) {
  if (e instanceof GraphRecursionError) {
    console.log("Recursion limit reached.");
  } else {
    throw e;
  }
}

Recursion limit reached.
